In [85]:
import param as pm
import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv

In [145]:
class Uniswap(pm.Parameterized):
    K = pm.Number(constant=True, bounds=(0, None))
    L = pm.Number(softbounds=(1, 100), bounds=(0, None), step=1)
    x = pm.Number(20, softbounds=(1, 100), bounds=(0, None), step=1)
    y = pm.Number(80, softbounds=(1, 100), bounds=(0, None), step=1)
    price_x_y = pm.Number(constant=True)
    execution_price_x_y = pm.Number(constant=True)
    x_in = pm.Number(1, softbounds=(-100,100), step=0.1)
    y_out = pm.Number(constant=True)
    switch_xy = pm.Action(lambda self: self._switch_xy())
    swap = pm.Action(lambda self: self._swap())
    _updating = pm.Boolean(False, precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.update_xy()
        self.update_x_in()
    
    @pm.depends('x', 'y', watch=True)
    def update_xy(self):
        if not self._updating:
            self._updating = True
            with pm.edit_constant(self):
                self.K = self.x * self.y
                self.L = np.sqrt(self.K)
                self.price_x_y = self.price_yx(self.x)
            self.param['x_in'].bounds = [-self.x+1,None]
            self.x_in = self.param['x_in'].crop_to_bounds(self.x_in)
            self._updating = False
            
    @pm.depends('L', watch=True)
    def update_L(self):
        if not self._updating:
            self._updating = True
            with pm.edit_constant(self):
                self.K = self.L**2
            new_y = np.sqrt((self.y / self.x) * self.K)
            new_x = np.sqrt((self.x / self.y) * self.K)
            self.y = new_y
            self.x = new_x
            self._updating = False
            
    @pm.depends('x_in', 'L', watch=True)
    def update_x_in(self):
        with pm.edit_constant(self):
            self.y_out = self.y - self.liquidity_yx(self.x+self.x_in)
            if self.x_in == 0:
                self.execution_price_x_y = self.price_x_y
            else:
                self.execution_price_x_y = self.y_out / self.x_in
            
    def _switch_xy(self):
        new_y = self.x
        new_x = self.y
        new_x_in = -self.y_out
        with pm.parameterized.batch_call_watchers(self):
            self.y = new_y
            self.x = new_x
            self.param['x_in'].bounds = [-self.x+1,None]
            self.x_in = self.param['x_in'].crop_to_bounds(self.x_in)
            self.x_in = new_x_in
            
    def _swap(self):
        new_y = self.y - self.y_out
        new_x = self.x + self.x_in
        with pm.parameterized.discard_events(self):
            self.y = new_y
            self.x = new_x
        self.param.trigger('y','x')
            
    def liquidity_yx(self, x):
        return self.K / x
    
    def price_yx(self, x):
        return self.K / x**2
    
    def xy_curve(self):
        xs = np.linspace(*self.param['x'].softbounds, num=1000)
        ys = self.liquidity_yx(xs)
        df = pd.DataFrame({'x':xs,'y':ys})
        return df
    
    def view_curves(self):
        xy_curve = self.xy_curve().hvplot.line(x='x',y='y', label='xy=k')
        xy_curve.opts( 
            color='purple', 
            xlim=self.param['x'].softbounds, 
            ylim=self.param['y'].softbounds,
        )
        return xy_curve
    
    def view_points(self):
        ll = (self.L, self.L, 'Liquidity')
        xy_start = (self.x, self.y, 'xy')
        xy_end   = (self.x+self.x_in, self.liquidity_yx(self.x+self.x_in), 'xy_end')
        points = pd.DataFrame([ll, xy_start, xy_end], columns=['x','y','label']).hvplot.scatter(
            x='x',
            y='y',
            by='label',
            color=['purple', 'green', 'orange'],
            size=120,)        
        return points
    
    @pm.depends('L', 'x_in')
    def view_chart(self):
        points = self.view_points()
        curves = self.view_curves()
        chart = curves * points
        chart.opts(
            title="Uniswap Math",
            width=580,
            height=640,
            legend_position="top_right",
        )
        return chart

    def view(self):
        return pn.Row(self, self.view_chart)
    
    def reset(self):
        with pm.edit_constant(self):
            for param in self.param:
                if param not in ["name"]:
                    setattr(self, param, self.param[param].default)

In [148]:
pn.config.throttled = False

uniswap = Uniswap()
uniswap.view()

Row
    [0] Column(margin=5, name='Uniswap', width=300)
        [0] StaticText(value='<b>Uniswap</b>')
        [1] FloatInput(disabled=True, name='K', start=0, value=1600)
        [2] FloatSlider(end=100, name='L', start=1, step=1, value=40.0)
        [3] FloatSlider(end=100, name='X', start=1, step=1, value=20)
        [4] FloatSlider(end=100, name='Y', start=1, step=1, value=80)
        [5] FloatInput(disabled=True, name='Price x y', value=4.0)
        [6] FloatInput(disabled=True, name='Execution price x y', value=3.80952380952381)
        [7] FloatSlider(end=100, name='X in', start=-19, value=1)
        [8] FloatInput(disabled=True, name='Y out', value=3.80952380952381)
        [9] Button(name='Switch xy')
        [10] Button(name='Swap')
    [1] ParamMethod(method, _pane=HoloViews)

### Experiment #0: Heatmap of y_out with constant x_in.

In [124]:
pn.config.throttled = True
uniswap.reset()
num = 5
xs = np.linspace(*uniswap.param['x'].softbounds, num=num)
ys = np.linspace(*uniswap.param['y'].softbounds, num=num)
X, Y = np.meshgrid(xs, ys)
phase_space = pd.DataFrame(np.stack([X.ravel(), Y.ravel()]).T, columns=['x','y'])

In [125]:
phase_space['y_out'] = phase_space.apply(lambda row: uniswap.y_out if uniswap.param.update(**row) is None else None, axis=1)

In [126]:
phase_space

,x,y,y_out
0,1.00,1.00,0.500000
1,25.75,1.00,0.037383
2,50.50,1.00,0.019417
3,75.25,1.00,0.013115
4,100.00,1.00,0.009901
5,1.00,25.75,12.875000
6,25.75,25.75,0.962617
7,50.50,25.75,0.500000
8,75.25,25.75,0.337705
9,100.00,25.75,0.254950


In [127]:
hv.HeatMap(phase_space, kdims=['x','y'], vdims=['y_out']).opts(height=600, width=600, title='y_out Heatmap', colorbar=True)

:HeatMap   [x,y]   (y_out)

### Experiment #1: Observing mechanism data according to Liquidity of the pair (L)

In [139]:
pn.config.throttled = True
uniswap.reset()
drop_columns=['_updating', 'name', 'swap', 'switch_xy']
experiment1_columns = ['L', 'execution_price_x_y', 'price_x_y', 'x', 'x_in', 'y', 'y_out']

liquidity_range = np.linspace(*uniswap.param['L'].softbounds, num=50)

data = []
for L in liquidity_range:
    uniswap.L = L
    data.append(uniswap.param.values())
    
df = pd.DataFrame(data).drop(drop_columns, axis=1)
df = df[experiment1_columns]

In [140]:
df.head()

,L,execution_price_x_y,price_x_y,x,x_in,y,y_out
0,1.000000,1.333333,0.0,0.500000,1,2.000000,1.333333
1,3.020408,2.406504,0.0,1.510204,1,6.040816,2.406504
2,5.040816,2.863768,0.0,2.520408,1,10.081633,2.863768
3,7.061224,3.117117,0.0,3.530612,1,14.122449,3.117117
4,9.081633,3.278085,0.0,4.540816,1,18.163265,3.278085


In [141]:
df.hvplot.line(y=['execution_price_x_y'], x='L')

:Curve   [L]   (execution_price_x_y)

### Experiment #2: Calculating Slippage [WIP]    def _reset(self):
        for param in self.param:
            if param not in ["name"]:
                setattr(self, param, self.param[param].default)

ChatGPT provided the following formulas for slippage:

    Percentage slippage = (Execution price - Expected price) / Expected price * 100%

    Basis point slippage = (Execution price - Expected price) / Expected price * 10,000 basis points

    Dollar slippage = (Execution price - Expected price) * Quantity
    
I asked the chat bot to proved python functions for the above, and I received the following:

In [13]:
def percentage_slippage(execution_price, expected_price):
    return ((execution_price - expected_price) / expected_price) * 100


def basis_point_slippage(execution_price, expected_price):
    return ((execution_price - expected_price) / expected_price) * 10000


def dollar_slippage(execution_price, expected_price, quantity):
    return (execution_price - expected_price) * quantity


Simple slippage calculation inspired by the above.

In [14]:
# Slippage
df['price_slippage_x_y'] = np.abs(df['price_x_y'] - df['execution_price_x_y'])
df['percentage_slippage_x_y'] = (df['price_slippage_x_y'] / df['price_x_y']) * 100
df['absolute_slippage_x_y'] = df['price_slippage_x_y'] * df['x_in']

In [15]:
df.head()

,L,execution_price_x_y,execution_price_y_x,price_x_y,price_y_x,x,x_in,y,y_out,price_slippage_x_y,percentage_slippage_x_y,absolute_slippage_x_y
0,1.000000,0.500000,2.000000,1.0,1.0,1.000000,1,1.000000,0.500000,0.500000,50.000000,0.500000
1,3.020408,0.751269,1.331081,1.0,1.0,3.020408,1,3.020408,0.751269,0.248731,24.873096,0.248731
2,5.040816,0.834459,1.198381,1.0,1.0,5.040816,1,5.040816,0.834459,0.165541,16.554054,0.165541
3,7.061224,0.875949,1.141618,1.0,1.0,7.061224,1,7.061224,0.875949,0.124051,12.405063,0.124051
4,9.081633,0.900810,1.110112,1.0,1.0,9.081633,1,9.081633,0.900810,0.099190,9.919028,0.099190


#### 2.a Defining Slippage

In [16]:
df['percentage_slippage'] = df[['execution_price_x_y', 'price_x_y']].apply(lambda row: percentage_slippage(*row), axis=1)
df['basis_point_slippage'] = df[['execution_price_x_y', 'price_x_y']].apply(lambda row: basis_point_slippage(*row), axis=1)
df['dollar_slippage'] = df[['execution_price_x_y', 'price_x_y', 'x_in']].apply(lambda row: dollar_slippage(*row), axis=1)

In [17]:
df

,L,execution_price_x_y,execution_price_y_x,price_x_y,price_y_x,x,x_in,y,y_out,price_slippage_x_y,percentage_slippage_x_y,absolute_slippage_x_y,percentage_slippage,basis_point_slippage,dollar_slippage
0,1.000000,0.500000,2.000000,1.0,1.0,1.000000,1,1.000000,0.500000,0.500000,50.000000,0.500000,-50.000000,-5000.000000,-0.500000
1,3.020408,0.751269,1.331081,1.0,1.0,3.020408,1,3.020408,0.751269,0.248731,24.873096,0.248731,-24.873096,-2487.309645,-0.248731
2,5.040816,0.834459,1.198381,1.0,1.0,5.040816,1,5.040816,0.834459,0.165541,16.554054,0.165541,-16.554054,-1655.405405,-0.165541
3,7.061224,0.875949,1.141618,1.0,1.0,7.061224,1,7.061224,0.875949,0.124051,12.405063,0.124051,-12.405063,-1240.506329,-0.124051
4,9.081633,0.900810,1.110112,1.0,1.0,9.081633,1,9.081633,0.900810,0.099190,9.919028,0.099190,-9.919028,-991.902834,-0.099190
5,11.102041,0.917369,1.090074,1.0,1.0,11.102041,1,11.102041,0.917369,0.082631,8.263069,0.082631,-8.263069,-826.306914,-0.082631
6,13.122449,0.929191,1.076205,1.0,1.0,13.122449,1,13.122449,0.929191,0.070809,7.080925,0.070809,-7.080925,-708.092486,-0.070809
7,15.142857,0.938053,1.066038,1.0,1.0,15.142857,1,15.142857,0.938053,0.061947,6.194690,0.061947,-6.194690,-619.469027,-0.061947
8,17.163265,0.944944,1.058264,1.0,1.0,17.163265,1,17.163265,0.944944,0.055056,5.505618,0.055056,-5.505618,-550.561798,-0.055056
9,19.183673,0.950455,1.052128,1.0,1.0,19.183673,1,19.183673,0.950455,0.049545,4.954499,0.049545,-4.954499,-495.449949,-0.049545


#### 2.b Calculating Slippage in the Context of a Bonding Curve

The following code is generated by chatGPT to calculate slippage in the context of a uniswap constant product market.

In [18]:
# Example code for computing expected price, execution price, and quantity for a Uniswap pair

import math

# Define pair properties and transaction details
reserve_1 = 1000.0
reserve_2 = 2000.0
amount_1 = 100.0
fee = 0.003

# Calculate expected price based on reserves and transaction details
expected_price = reserve_2 / reserve_1
expected_price *= 1 - fee if amount_1 > 0 else 1 + fee
expected_price_new = (reserve_2 + amount_1) / (reserve_1 - amount_1)
expected_price_new *= 1 - fee if amount_1 > 0 else 1 + fee
expected_price = math.sqrt(expected_price * expected_price_new)

# Calculate execution price based on reserves and transaction details
execution_price = reserve_2 / (reserve_1 - amount_1) if amount_1 > 0 else (reserve_2 + amount_1) / reserve_1

# Calculate quantity based on reserves and transaction details
quantity = amount_1 / (1 + fee) if amount_1 > 0 else (reserve_1 - reserve_1 / (reserve_2 - amount_1)) / (1 + fee)

# Print results
print(f"Expected price: {expected_price}")
print(f"Execution price: {execution_price}")
print(f"Quantity: {quantity}")

Expected price: 2.153766158770879
Execution price: 2.2222222222222223
Quantity: 99.70089730807578


#### 3.c Mapping Terms

Let's map our terms to the above terms as an excercise to see what data we have.

In [19]:
df['reserve_1'] = df['x']
df['reserve_2'] = df['y']
df['amount_1'] = df['x_in']
df['fee'] = 0.003

To process the above data, let's practice on toy data first.

#### 3.d Vectorizing ChatGPT's Formula for Slippage on Toy Data

In [20]:
toy_df = pd.DataFrame([
    dict(
        reserve_1 = 1000.0,
        reserve_2 = 2000.0,
        amount_1 = 100.0,
        fee = 0.003,
    )])

3.d i) By processing a dataframe.

In [21]:
def vectorized_slippage_formula(df):
    
    # Calculate expected price based on reserves and transaction details

    # Fee factor
    df['fee_factor'] = np.select(condlist=df['amount_1'] > 0, choicelist=1 - df['fee'], default=1 + df['fee'])
    
    # First expected price
    df['expected_price_first'] = (df['reserve_2'] / df['reserve_1']) 
    df['expected_price_first_with_fee'] = df['expected_price_first'] * df['fee_factor']
    
    # New expected price
    df['expected_price_new'] = ((df['reserve_2'] + df['amount_1']) / (df['reserve_1'] - df['amount_1'])) 
    df['expected_price_new_with_fee'] = df['expected_price_new'] * df['fee_factor']
    
    # Final expected price
    df['expected_price_final'] = np.sqrt(df['expected_price_first'] * df['expected_price_new'])
    df['expected_price_final_with_fee'] = np.sqrt(df['expected_price_first_with_fee'] * df['expected_price_new_with_fee'])

    # Calculate execution price based on reserves and transaction details
    df['execution_price'] = np.select(
        condlist=df['amount_1'] > 0, 
        choicelist=df['reserve_2'] / (df['reserve_1'] - df['amount_1']), 
        default=(df['reserve_2'] + df['amount_1']) / df['reserve_1'],
    )
    
    # Calculate quantity based on reserves and transaction details
    df['quantity'] = np.select(
        condlist=df['amount_1'] > 0, 
        choicelist=df['amount_1'] / (1 + df['fee']), 
        default=(df['reserve_1'] - df['reserve_1'] / (df['reserve_2'] - df['amount_1'])) / (1 + df['fee']),
    )

    return df

vectorized_slippage_formula(toy_df)

,reserve_1,reserve_2,amount_1,fee,fee_factor,expected_price_first,expected_price_first_with_fee,expected_price_new,expected_price_new_with_fee,expected_price_final,expected_price_final_with_fee,execution_price,quantity
0,1000.0,2000.0,100.0,0.003,0.997,2.0,1.994,2.333333,2.326333,2.160247,2.153766,2.222222,99.700897


3.d ii) By vectorizing functions.

In [22]:
# Example code for computing expected price, execution price, and quantity for a Uniswap pair

transaction_example = dict(
    reserve_1 = 1000.0,
    reserve_2 = 2000.0,
    amount_1 = 100.0,
    fee = 0.003,
)

# Define pair properties and transaction details
def expected_price(**transaction):
    # Calculate expected price based on reserves and transaction details
    expected_price = reserve_2 / reserve_1
    expected_price *= 1 - fee if amount_1 > 0 else 1 + fee
    expected_price_new = (reserve_2 + amount_1) / (reserve_1 - amount_1)
    expected_price_new *= 1 - fee if amount_1 > 0 else 1 + fee
    expected_price = math.sqrt(expected_price * expected_price_new)
    print(transaction)
    print(expected_price)
    return expected_price

def execution_price(**transaction):
    # Calculate execution price based on reserves and transaction details
    execution_price = reserve_2 / (reserve_1 - amount_1) if amount_1 > 0 else (reserve_2 + amount_1) / reserve_1
    return execution_price

def quantity(**transaction):
    # Calculate quantity based on reserves and transaction details
    quantity = amount_1 / (1 + fee) if amount_1 > 0 else (reserve_1 - reserve_1 / (reserve_2 - amount_1)) / (1 + fee)
    return quantity

# Print results
print(f"Expected price: {expected_price(**transaction_example)}")
print(f"Execution price: {execution_price(**transaction_example)}")
print(f"Quantity: {quantity(**transaction_example)}")

{'reserve_1': 1000.0, 'reserve_2': 2000.0, 'amount_1': 100.0, 'fee': 0.003}
2.153766158770879
Expected price: 2.153766158770879
Execution price: 2.2222222222222223
Quantity: 99.70089730807578


In [23]:
toy_df['expected_price'] = toy_df[['reserve_1', 'reserve_2', 'amount_1', 'fee']].apply(lambda row: expected_price(**row), axis=1)
toy_df['execution_price'] = toy_df[['reserve_1', 'reserve_2', 'amount_1', 'fee']].apply(lambda row: execution_price(**row), axis=1)
toy_df['quantity'] = toy_df[['reserve_1', 'reserve_2', 'amount_1', 'fee']].apply(lambda row: quantity(**row), axis=1)

{'reserve_1': 1000.0, 'reserve_2': 2000.0, 'amount_1': 100.0, 'fee': 0.003}
2.153766158770879


In [24]:
toy_df

,reserve_1,reserve_2,amount_1,fee,fee_factor,expected_price_first,expected_price_first_with_fee,expected_price_new,expected_price_new_with_fee,expected_price_final,expected_price_final_with_fee,execution_price,quantity,expected_price
0,1000.0,2000.0,100.0,0.003,0.997,2.0,1.994,2.333333,2.326333,2.160247,2.153766,2.222222,99.700897,2.153766


#### 3.e Applying functions to our mapped data.

In [25]:
# vectorized_slippage_formula(df).bfill().rename({'expected_price_final_with_fee': 'expected_price'},axis=1).drop(['expected_price_first', 'expected_price_first_with_fee', 'expected_price_new', 'expected_price_new_with_fee', 'expected_price_with_fee', 'expected_price_final'], axis=1)

In [26]:
expected_price(**{'reserve_1': 0.5, 'reserve_2': 2.0, 'amount_1': 1.0, 'fee': 0.003})

{'reserve_1': 0.5, 'reserve_2': 2.0, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879


2.153766158770879

In [27]:
expected_price(**{'reserve_1': 1.510204081632653, 'reserve_2': 6.040816326530612, 'amount_1': 1.0, 'fee': 0.003})

{'reserve_1': 1.510204081632653, 'reserve_2': 6.040816326530612, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879


2.153766158770879

In [28]:
df['expected_price'] = df[['reserve_1', 'reserve_2', 'amount_1', 'fee']].apply(lambda row: expected_price(**row), axis=1)
df['execution_price'] = df[['reserve_1', 'reserve_2', 'amount_1', 'fee']].apply(lambda row: execution_price(**row), axis=1)
df['quantity'] = df[['reserve_1', 'reserve_2', 'amount_1', 'fee']].apply(lambda row: quantity(**row), axis=1)

{'reserve_1': 1.0, 'reserve_2': 1.0, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 3.020408163265306, 'reserve_2': 3.020408163265306, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 5.040816326530612, 'reserve_2': 5.040816326530612, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 7.061224489795918, 'reserve_2': 7.061224489795918, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 9.081632653061224, 'reserve_2': 9.081632653061224, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 11.102040816326529, 'reserve_2': 11.102040816326529, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 13.122448979591836, 'reserve_2': 13.122448979591836, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 15.142857142857142, 'reserve_2': 15.142857142857142, 'amount_1': 1.0, 'fee': 0.003}
2.153766158770879
{'reserve_1': 17.163265306122447, 'reserve_2': 17.163265306122447, 'amount_1': 1.0, 'fee': 0.003}
2.1537661587